MIT Election Data and Science Lab, 2018, "County Presidential Election Returns 2000-2016"

US Census

In [130]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns

from dotenv import load_dotenv
from functools import reduce
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from xgboost import XGBClassifier, XGBRegressor, plot_importance

%matplotlib inline
plt.style.use("ggplot")
sns.set_context("talk")

In [28]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [121]:
variables = pd.DataFrame(requests.get(
    "https://api.census.gov/data/2017/acs/acs5/profile/variables.json").json()["variables"].keys(),
                         columns=["index_name"])

In [161]:
variables_selected = variables.loc[((~variables.index_name.str.contains(r'DP')|
                                     (variables.index_name.str.contains(r'DP')&
                                      variables.index_name.str.contains(r'[0-9]PE')))&
                                    ~variables.index_name.str.contains(r'PR_'))]

In [162]:
len(variables_selected)

558

In [222]:
resp = requests.get(
    "https://api.census.gov/data/2017/acs/acs5/profile?get=NAME&for=county:*&in=state:*&key={}"
    .format(censuskey))

In [223]:
county = pd.DataFrame(resp.json()[1:], columns=resp.json()[0])

In [226]:
county

,NAME,state,county
0,"Pickens County, Alabama",01,107
1,"Sumter County, Alabama",01,119
2,"Jefferson County, Alabama",01,073
3,"Choctaw County, Alabama",01,023
4,"Franklin County, Alabama",01,059
5,"Monroe County, Alabama",01,099
6,"DeKalb County, Alabama",01,049
7,"Escambia County, Alabama",01,053
8,"Lamar County, Alabama",01,075
9,"Randolph County, Alabama",01,111


In [157]:
varlist = []
for var in variables_selected.index_name:
    resp = requests.get(
        "https://api.census.gov/data/2017/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
        .format(var, censuskey))
    if resp.status_code ==200:
        varlist.append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))

In [158]:
df_merged = reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist)

In [177]:
df_merged.dropna(axis=1, inplace=True)

In [181]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220 entries, 0 to 3219
Columns: 375 entries, DP05_0050PE to DP04_0095PE
dtypes: object(375)
memory usage: 9.2+ MB


In [431]:
df_clean = pd.concat([df_merged.loc[:, ~df_merged.columns.str.contains(r'[0-9]PE')].
                    drop(columns=["STATE", "COUNTY"]),
                    df_merged.loc[:, df_merged.columns.str.contains(r'[0-9]PE')]
                    .loc[:, ((df_merged.loc[:, df_merged.columns.str.contains(r'[0-9]PE')].max()<100)&
                             (df_merged.loc[:, df_merged.columns.str.contains(r'[0-9]PE')].min()>=0))]],
                     axis=1
                   )

In [432]:
df_clean = pd.merge(county, df_clean, on = ["state", "county"])

In [433]:
df_clean.county = df_clean.NAME.str.split(',', expand=True
                                         ).loc[:, 0].replace({" County":"",
                                                              " Municipio":"",
                                                              " city": "",
                                                              " Parish":""
                                                             }, regex=True)
df_clean.state = df_clean.NAME.str.split(',', expand=True).loc[:, 1].map(
    lambda x: x.lstrip(" "))
df_clean.iloc[1149, 2] = "Baltimore City"
df_clean.iloc[1530, 2] = "St. Louis County"
df_clean.iloc[1570, 2] = "St. Louis City"
df_clean.head()

,NAME,state,county,GEO_ID,GEOCOMP,DP05_0050PE,DP03_0048PE,DP04_0029PE,DP03_0022PE,DP04_0003PE,...,DP04_0140PE,DP05_0010PE,DP04_0118PE,DP05_0054PE,DP04_0051PE,DP05_0041PE,DP04_0082PE,DP04_0105PE,DP03_0115PE,DP03_0039PE
0,"Pickens County, Alabama",Alabama,Pickens,0500000US01107,00,0.0,18.1,0.9,1.6,20.1,...,12.8,12.6,43.9,0.0,4.7,0.0,31.7,19.7,89.6,0.8
1,"Sumter County, Alabama",Alabama,Sumter,0500000US01119,00,0.0,23.6,3.2,3.1,25.8,...,4.9,11.3,38.4,0.0,8.0,0.0,32.5,19.1,70.6,1.1
2,"Jefferson County, Alabama",Alabama,Jefferson,0500000US01073,00,0.3,13.5,1.4,1.4,14.6,...,11.4,14.4,47.6,0.0,11.1,0.0,23.1,30.7,83.7,2.3
3,"Choctaw County, Alabama",Alabama,Choctaw,0500000US01023,00,0.0,14.8,0.5,1.7,25.5,...,9.3,9.9,38.4,0.0,5.7,0.0,29.0,26.1,82.9,1.6
4,"Franklin County, Alabama",Alabama,Franklin,0500000US01059,00,0.0,15.6,1.5,0.2,18.4,...,9.1,12.1,43.1,0.0,8.1,0.0,36.0,23.8,76.0,2.7


In [442]:
df_clean.loc[df_clean.state=="Missouri"].county.unique()

array(['Callaway', 'Marion', 'Montgomery', 'Osage', 'Clinton', 'Cooper',
       'Dallas', 'Ripley', 'Stoddard', 'Warren', 'Washington', 'Gentry',
       'Hickory', 'Macon', 'Morgan', 'Wayne', 'Bollinger',
       'Cape Girardeau', 'McDonald', 'New Madrid', 'Perry', 'Moniteau',
       'Shelby', 'Webster', 'Lincoln', 'Mississippi', 'Audrain',
       'Christian', 'Monroe', 'Newton', 'Ozark', 'DeKalb', 'Greene',
       'Oregon', 'Pemiscot', 'Atchison', 'Polk', 'Ralls', 'Ray', 'Pettis',
       'Ste. Genevieve', 'Shannon', 'Bates', 'Benton', 'Mercer',
       'Madison', 'Reynolds', 'St. Louis County', 'Randolph', 'Douglas',
       'Carter', 'Dade', 'Linn', 'Scott', 'Stone', 'Barry', 'Pike',
       'St. Charles', 'Dunklin', 'Franklin', 'Henry', 'Howell', 'Johnson',
       'Laclede', 'Howard', 'Jackson', 'Lawrence', 'Maries', 'Phelps',
       'Platte', 'St. Francois', 'Miller', 'Pulaski', 'Putnam', 'Taney',
       'Harrison', 'Lafayette', 'St. Clair', 'Wright', 'Clay', 'Vernon',
       'Adair', 

In [393]:
election = pd.read_csv("../data/countypres_2000-2016.csv")

In [394]:
election.dropna(inplace=True)

In [395]:
election["candidateshare"] = election.candidatevotes / election.totalvotes
hillary = election.loc[((election.year==2016)&(election.candidate=="Hillary Clinton"))]
hillary = hillary[["state", "county", "candidateshare"]]
hillary.rename(columns={"candidateshare":"hillary"}, inplace=True)
trump = election.loc[((election.year==2016)&(election.candidate=="Donald Trump"))]
trump = trump[["state", "county", "candidateshare"]]
trump.rename(columns={"candidateshare":"trump"}, inplace=True)


In [396]:
election_result = pd.merge(hillary, trump, on=["state", "county"])

In [397]:
election_result["target"] = (election_result.trump > election_result.hillary)*1
election_result.head()

,state,county,hillary,trump,target
0,Alabama,Autauga,0.237697,0.727666,1
1,Alabama,Baldwin,0.193856,0.765457,1
2,Alabama,Barbour,0.465278,0.520967,1
3,Alabama,Bibb,0.212496,0.764032,1
4,Alabama,Blount,0.084258,0.893348,1


In [398]:
election_result.describe()

,hillary,trump,target
count,3161.000000,3161.000000,3161.000000
mean,0.316797,0.630046,0.839608
std,0.152992,0.158204,0.367028
min,0.031447,0.040875,0.000000
25%,0.204164,0.540706,1.000000
50%,0.284328,0.661863,1.000000
75%,0.402042,0.747770,1.000000
max,0.908638,0.960334,1.000000


In [414]:
election_result.loc[((election_result.state=="Louisiana")
                     &(election_result.county=="La Salle")), "county"] = "LaSalle"
election_result.loc[((election_result.state=="Florida")
                     &(election_result.county=="Desoto")), "county"] = "DeSoto"
election_result.loc[((election_result.state=="Minnesota")
                     &(election_result.county=="Lac Qui Parle")), "county"] = "Lac qui Parle"
election_result.loc[((election_result.state=="Minnesota")
                     &(election_result.county=="Saint Louis")), "county"] = "St. Louis"
election_result.loc[((election_result.state=="New Mexico")
                     &(election_result.county=="Dona Ana")), "county"] = "Doña Ana"
election_result.loc[((election_result.state=="Texas")
                     &(election_result.county=="Dewitt")), "county"] = "DeWitt"

In [462]:
df = pd.merge(df_clean, election_result, on=["state", "county"], how="inner").apply(pd.to_numeric, errors="ignore")

In [463]:
df.head()

,NAME,state,county,GEO_ID,GEOCOMP,DP05_0050PE,DP03_0048PE,DP04_0029PE,DP03_0022PE,DP04_0003PE,...,DP05_0054PE,DP04_0051PE,DP05_0041PE,DP04_0082PE,DP04_0105PE,DP03_0115PE,DP03_0039PE,hillary,trump,target
0,"Pickens County, Alabama",Alabama,Pickens,0500000US01107,0,0.0,18.1,0.9,1.6,20.1,...,0.0,4.7,0.0,31.7,19.7,89.6,0.8,0.416265,0.571788,1
1,"Sumter County, Alabama",Alabama,Sumter,0500000US01119,0,0.0,23.6,3.2,3.1,25.8,...,0.0,8.0,0.0,32.5,19.1,70.6,1.1,0.740290,0.246607,0
2,"Jefferson County, Alabama",Alabama,Jefferson,0500000US01073,0,0.3,13.5,1.4,1.4,14.6,...,0.0,11.1,0.0,23.1,30.7,83.7,2.3,0.515706,0.443037,0
3,"Choctaw County, Alabama",Alabama,Choctaw,0500000US01023,0,0.0,14.8,0.5,1.7,25.5,...,0.0,5.7,0.0,29.0,26.1,82.9,1.6,0.426358,0.563083,1
4,"Franklin County, Alabama",Alabama,Franklin,0500000US01059,0,0.0,15.6,1.5,0.2,18.4,...,0.0,8.1,0.0,36.0,23.8,76.0,2.7,0.182475,0.786213,1


In [465]:
X = df.iloc[:, 5:-3]
y = df.target

In [478]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [479]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [490]:
rfc = RandomForestClassifier()
rfc.fit(X_train_scaled, y_train)
print("train: ", rfc.score(X_train_scaled, y_train))
print("test: ", rfc.score(X_test_scaled, y_test))

train:  0.9936224489795918
test:  0.9413265306122449


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [503]:
pd.DataFrame(rfc.feature_importances_, 
             index=X.columns, 
             columns=["importance"]
            ).sort_values(by="importance", ascending=False).head(20)

,importance
DP05_0077PE,0.060504
DP05_0037PE,0.049178
DP05_0064PE,0.047249
DP05_0044PE,0.037950
DP05_0046PE,0.035285
DP04_0014PE,0.029670
DP05_0038PE,0.028650
DP04_0046PE,0.027922
DP05_0065PE,0.024049
DP04_0083PE,0.020717


In [489]:
svc = SVC(probability=True)
svc.fit(X_train_scaled, y_train)
print("train: ", svc.score(X_train_scaled, y_train))
print("test: ", svc.score(X_test_scaled, y_test))

train:  0.969812925170068
test:  0.9540816326530612


In [506]:
xgc = XGBClassifier()
xgc.fit(X_train_scaled, y_train)
print("train: ", xgc.score(X_train_scaled, y_train))
print("test: ", xgc.score(X_test_scaled, y_test))

train:  0.985969387755102
test:  0.9528061224489796


In [509]:
pd.DataFrame(xgc.feature_importances_, 
             index=X.columns, 
             columns=["importance"]
            ).sort_values(by="importance", ascending=False).head(20)

,importance
DP03_0021PE,0.043544
DP05_0037PE,0.039449
DP05_0046PE,0.033600
DP04_0104PE,0.030905
DP04_0013PE,0.030835
DP05_0067PE,0.030052
DP05_0077PE,0.027235
DP03_0030PE,0.026092
DP04_0047PE,0.025154
DP04_0118PE,0.017549
